In [1]:
# Cell 1: Imports
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
import math

In [2]:
# Cell 2: Create directories
os.makedirs('result_shuffled/plots', exist_ok=True)
os.makedirs('result_shuffled/plots/individual_clients', exist_ok=True)

In [3]:
# Cell 3: Configuration
# ============================================
# FEDERATED LEARNING PARAMETERS
# ============================================
NUM_CLIENTS = 5      # 5-split data
NUM_ROUNDS = 10       # Number of communication rounds
BATCH_SIZE = 64

# ADAPTIVE TRAINING PARAMETERS
MIN_EPOCHS = 5       # Minimum epochs to train per round
MAX_EPOCHS = 100      # Maximum epochs per round (safety limit)
PATIENCE = 50        # Stop if no improvement for this many epochs
TARGET_IMPROVEMENT = 0.001  # Minimum accuracy improvement required (0.1%)

print("=" * 60)
print("FEDERATED LEARNING CONFIGURATION (ADAPTIVE)")
print("=" * 60)
print(f"Number of Clients: {NUM_CLIENTS}")
print(f"Communication Rounds: {NUM_ROUNDS}")
print(f"Min Epochs per Round: {MIN_EPOCHS}")
print(f"Max Epochs per Round: {MAX_EPOCHS}")
print(f"Patience: {PATIENCE}")
print(f"Target Improvement: {TARGET_IMPROVEMENT*100:.2f}%")
print(f"Batch Size: {BATCH_SIZE}")
print("=" * 60 + "\n")

FEDERATED LEARNING CONFIGURATION (ADAPTIVE)
Number of Clients: 5
Communication Rounds: 10
Min Epochs per Round: 5
Max Epochs per Round: 100
Patience: 50
Target Improvement: 0.10%
Batch Size: 64



In [4]:
# Cell 4: Load and normalize client data
# ============================================
# LOAD AND NORMALIZE CLIENT DATA
# ============================================
print("=" * 60)
print("LOADING AND NORMALIZING CLIENT DATA")
print("=" * 60)

client_data = []
for i in range(1, NUM_CLIENTS + 1):
    data_path = f'data/5_split_shuffled1/mnist_part{i}.npz'
    
    # Load .npz file
    data = np.load(data_path)
    
    # Get the data
    x_train = data['x_train']
    y_train = data['y_train']
    x_test = data['x_test']
    y_test = data['y_test']
    
    # Normalize
    x_train = x_train / 255
    x_test = x_test / 255
    
    # Flatten
    x_train_flatten = x_train.reshape(len(x_train), 28*28)
    x_test_flatten = x_test.reshape(len(x_test), 28*28)
    
    # Store normalized and flattened data
    normalized_data = {
        'x_train': x_train_flatten,
        'y_train': y_train,
        'x_test': x_test_flatten,
        'y_test': y_test
    }
    
    client_data.append(normalized_data)
    print(f"✓ Client {i} - Train: {len(x_train_flatten)}, Test: {len(x_test_flatten)}")

print(f"\n✓ Total clients loaded: {len(client_data)}\n")

LOADING AND NORMALIZING CLIENT DATA
✓ Client 1 - Train: 11999, Test: 2001
✓ Client 2 - Train: 11999, Test: 2001
✓ Client 3 - Train: 11999, Test: 2001
✓ Client 4 - Train: 11999, Test: 2001
✓ Client 5 - Train: 11999, Test: 2001

✓ Total clients loaded: 5



In [5]:
# Cell 5: Define model architecture
# ============================================
# DEFINE MODEL ARCHITECTURE
# ============================================
def create_model():
    """Model architecture"""
    model = keras.Sequential([
        # First hidden layer - Feature extraction
        keras.layers.Dense(128, input_shape=(784,), activation="relu"),
        keras.layers.BatchNormalization(), 
        keras.layers.Dropout(0.3),
        
        # Second hidden layer - Pattern recognition
        keras.layers.Dense(64, activation="relu"),
        keras.layers.BatchNormalization(), 
        keras.layers.Dropout(0.2), 
        
        # Third hidden layer - Feature refinement
        keras.layers.Dense(32, activation="relu"),
        keras.layers.BatchNormalization(), 
        keras.layers.Dropout(0.2), 
        
        # Output layer - Classification
        keras.layers.Dense(10, activation="softmax")
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0005),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

In [6]:
# Cell 6: Federated averaging function
# ============================================
# FEDERATED AVERAGING FUNCTION
# ============================================
def average_weights(weights_list):
    """Average weights from all clients"""
    averaged_weights = []
    for weights_tuple in zip(*weights_list):
        averaged_weights.append(np.mean(weights_tuple, axis=0))
    return averaged_weights

In [7]:
# Cell 7: Initialize global model and tracking variables
# ============================================
# INITIALIZE GLOBAL MODEL
# ============================================
print("Initializing global model...")
global_model = create_model()
global_model.summary()
global_weights = global_model.get_weights()
print("\n✓ Global model initialized\n")

# Track accuracy for each client
client_test_accuracies = {f'client_{i}': [] for i in range(NUM_CLIENTS)}

# Track previous round accuracy for each client (start with 0)
client_prev_accuracy = {f'client_{i}': 0.0 for i in range(NUM_CLIENTS)}

# Track epochs used per client per round
client_epochs_used = {f'client_{i}': [] for i in range(NUM_CLIENTS)}

# Store training histories for plotting
all_training_histories = []

Initializing global model...


d:\amrita intern\tfenv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 112,042 (437.66 KB)

 Trainable params: 111,594 (435.91 KB)

 Non-trainable params: 448 (1.75 KB)


✓ Global model initialized



In [8]:
# Cell 8: Federated learning training loop with ADAPTIVE EPOCHS
# ============================================
# FEDERATED LEARNING TRAINING LOOP - ADAPTIVE EPOCHS
# ============================================
print("=" * 60)
print("STARTING FEDERATED LEARNING (ADAPTIVE EPOCHS)")
print("=" * 60 + "\n")

for round_num in range(NUM_ROUNDS):
    print(f"\n{'='*70}")
    print(f"COMMUNICATION ROUND {round_num + 1}/{NUM_ROUNDS}")
    print(f"{'='*70}\n")
    
    local_weights = []
    round_histories = []
    
    # Train each client
    for client_id in range(NUM_CLIENTS):
        print(f"\n{'─'*70}")
        print(f"[Client {client_id + 1}] Starting adaptive local training...")
        print(f"[Client {client_id + 1}] Previous accuracy: {client_prev_accuracy[f'client_{client_id}']*100:.2f}%")
        print(f"[Client {client_id + 1}] Target: > {client_prev_accuracy[f'client_{client_id}']*100:.2f}%")
        print(f"{'─'*70}")
        
        # Create fresh model and load global weights
        client_model = create_model()
        client_model.set_weights(global_weights)
        
        # Get client's normalized data
        x_train = client_data[client_id]['x_train']
        y_train = client_data[client_id]['y_train']
        x_test = client_data[client_id]['x_test']
        y_test = client_data[client_id]['y_test']
        
        # Get previous accuracy target
        prev_acc = client_prev_accuracy[f'client_{client_id}']
        target_acc = prev_acc + TARGET_IMPROVEMENT
        
        # ADAPTIVE TRAINING: Train epoch by epoch
        best_accuracy = 0.0
        epochs_without_improvement = 0
        total_epochs = 0
        training_history = {'accuracy': [], 'val_accuracy': [], 'loss': [], 'val_loss': []}
        
        for epoch in range(MAX_EPOCHS):
            # Train for 1 epoch
            history = client_model.fit(
                x_train, y_train,
                epochs=1,
                batch_size=BATCH_SIZE,
                validation_split=0.1,
                verbose=0
            )
            
            total_epochs += 1
            
            # Store history
            training_history['accuracy'].append(history.history['accuracy'][0])
            training_history['val_accuracy'].append(history.history['val_accuracy'][0])
            training_history['loss'].append(history.history['loss'][0])
            training_history['val_loss'].append(history.history['val_loss'][0])
            
            # Evaluate on test set
            _, current_accuracy = client_model.evaluate(x_test, y_test, verbose=0)
            
            # Check if improved
            if current_accuracy > best_accuracy:
                best_accuracy = current_accuracy
                epochs_without_improvement = 0
            else:
                epochs_without_improvement += 1
            
            # Print progress every 5 epochs
            if (epoch + 1) % 5 == 0:
                print(f"   Epoch {epoch + 1}: Test Acc = {current_accuracy*100:.2f}% (Best: {best_accuracy*100:.2f}%)")
            
            # Check stopping conditions (after minimum epochs)
            if total_epochs >= MIN_EPOCHS:
                # Condition 1: Achieved target (better than previous round)
                if current_accuracy > target_acc:
                    print(f"   ✓ Target achieved at epoch {total_epochs}! ({current_accuracy*100:.2f}% > {target_acc*100:.2f}%)")
                    break
                
                # Condition 2: No improvement for PATIENCE epochs
                if epochs_without_improvement >= PATIENCE:
                    print(f"   ⚠ Early stopping at epoch {total_epochs} (no improvement for {PATIENCE} epochs)")
                    break
        
        # If reached max epochs
        if total_epochs >= MAX_EPOCHS:
            print(f"   ⚠ Reached max epochs ({MAX_EPOCHS})")
        
        # Create a history-like object for compatibility
        class HistoryObject:
            def __init__(self, history_dict):
                self.history = history_dict
        
        history_obj = HistoryObject(training_history)
        
        # Store history for this round
        round_histories.append({
            'client_id': client_id,
            'round': round_num,
            'history': history_obj,
            'epochs_used': total_epochs
        })
        
        # Final evaluation
        loss, final_accuracy = client_model.evaluate(x_test, y_test, verbose=0)
        
        # Update previous accuracy for next round
        client_prev_accuracy[f'client_{client_id}'] = final_accuracy
        
        # Store metrics
        client_test_accuracies[f'client_{client_id}'].append(final_accuracy)
        client_epochs_used[f'client_{client_id}'].append(total_epochs)
        
        print(f"\n{'─'*70}")
        print(f"🎯 Client {client_id + 1} Results:")
        print(f"   - Final Accuracy: {final_accuracy * 100:.2f}%")
        print(f"   - Epochs Used: {total_epochs}")
        print(f"   - Improvement: {(final_accuracy - prev_acc) * 100:+.2f}%")
        print(f"{'─'*70}\n")
        
        # Store weights for averaging
        local_weights.append(client_model.get_weights())
    
    # Store all histories for this round
    all_training_histories.append(round_histories)
    
    # Federated Averaging (Weight Exchange)
    print(f"\n{'🔄'*35}")
    print("WEIGHT EXCHANGE - FEDERATED AVERAGING")
    print(f"{'🔄'*35}")
    global_weights = average_weights(local_weights)
    global_model.set_weights(global_weights)
    print("✓ Global weights updated")
    
    # Print round summary
    round_epochs = [client_epochs_used[f'client_{i}'][-1] for i in range(NUM_CLIENTS)]
    round_accs = [client_test_accuracies[f'client_{i}'][-1] * 100 for i in range(NUM_CLIENTS)]
    print(f"\n📊 Round {round_num + 1} Summary:")
    print(f"   - Avg Epochs Used: {np.mean(round_epochs):.1f}")
    print(f"   - Avg Accuracy: {np.mean(round_accs):.2f}%")
    print(f"   - Min/Max Accuracy: {np.min(round_accs):.2f}% / {np.max(round_accs):.2f}%")
    print(f"{'🔄'*35}\n")

print("\n" + "="*70)
print("FEDERATED LEARNING COMPLETE!")
print("="*70 + "\n")

STARTING FEDERATED LEARNING (ADAPTIVE EPOCHS)


COMMUNICATION ROUND 1/10


──────────────────────────────────────────────────────────────────────
[Client 1] Starting adaptive local training...
[Client 1] Previous accuracy: 0.00%
[Client 1] Target: > 0.00%
──────────────────────────────────────────────────────────────────────
   Epoch 5: Test Acc = 89.06% (Best: 89.31%)
   ✓ Target achieved at epoch 5! (89.06% > 0.10%)

──────────────────────────────────────────────────────────────────────
🎯 Client 1 Results:
   - Final Accuracy: 89.06%
   - Epochs Used: 5
   - Improvement: +89.06%
──────────────────────────────────────────────────────────────────────


──────────────────────────────────────────────────────────────────────
[Client 2] Starting adaptive local training...
[Client 2] Previous accuracy: 0.00%
[Client 2] Target: > 0.00%
──────────────────────────────────────────────────────────────────────
   Epoch 5: Test Acc = 89.96% (Best: 89.96%)
   ✓ Target achieved at epoch 5! (89.96% >

In [9]:
# Cell 9: Display final results
# ============================================
# DISPLAY FINAL RESULTS
# ============================================
print("FINAL RESULTS:")
print("-" * 70)
print(f"{'Client':<10} {'Final Acc':<15} {'Total Epochs':<15} {'Avg Epochs/Round':<15}")
print("-" * 70)

for client_id in range(NUM_CLIENTS):
    final_acc = client_test_accuracies[f'client_{client_id}'][-1]
    total_epochs = sum(client_epochs_used[f'client_{client_id}'])
    avg_epochs = total_epochs / NUM_ROUNDS
    print(f"{client_id + 1:<10} {final_acc*100:.2f}%{'':<8} {total_epochs:<15} {avg_epochs:.1f}")

print("-" * 70)

# Overall summary
all_final_accs = [client_test_accuracies[f'client_{i}'][-1] * 100 for i in range(NUM_CLIENTS)]
all_total_epochs = [sum(client_epochs_used[f'client_{i}']) for i in range(NUM_CLIENTS)]

print(f"\n📊 OVERALL SUMMARY:")
print(f"   - Average Final Accuracy: {np.mean(all_final_accs):.2f}%")
print(f"   - Average Total Epochs: {np.mean(all_total_epochs):.1f}")
print(f"   - Total Training Epochs (all clients): {sum(all_total_epochs)}")
print("-" * 70 + "\n")

FINAL RESULTS:
----------------------------------------------------------------------
Client     Final Acc       Total Epochs    Avg Epochs/Round
----------------------------------------------------------------------
1          96.05%         187             18.7
2          95.25%         192             19.2
3          96.20%         129             12.9
4          96.45%         74              7.4
5          96.20%         87              8.7
----------------------------------------------------------------------

📊 OVERALL SUMMARY:
   - Average Final Accuracy: 96.03%
   - Average Total Epochs: 133.8
   - Total Training Epochs (all clients): 669
----------------------------------------------------------------------



In [10]:
# Cell 10: Generate all plots
# ============================================
# GENERATE PLOTS
# ============================================
print("=" * 70)
print("GENERATING PLOTS")
print("=" * 70)

# ────────────────────────────────────────────
# Plot 1: Individual graphs for each client
# ────────────────────────────────────────────
print("\n1. Creating individual client test accuracy plots...")

for client_id in range(NUM_CLIENTS):
    plt.figure(figsize=(10, 6))
    
    accuracies = [acc * 100 for acc in client_test_accuracies[f'client_{client_id}']]
    
    plt.plot(range(1, NUM_ROUNDS + 1), accuracies, 
             marker='o', linewidth=3, markersize=10, 
             color='blue', label=f'Client {client_id + 1}')
    
    # Add accuracy values on points
    for i, acc in enumerate(accuracies):
        plt.text(i + 1, acc + 0.3, f'{acc:.2f}%', 
                ha='center', fontsize=10, fontweight='bold')
    
    plt.title(f'Client {client_id + 1}: Test Accuracy vs Communication Rounds', 
              fontsize=16, fontweight='bold')
    plt.xlabel('Communication Round', fontsize=13)
    plt.ylabel('Test Accuracy (%)', fontsize=13)
    plt.legend(loc='lower right', fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.xticks(range(1, NUM_ROUNDS + 1))
    plt.ylim([85, 100])
    
    save_path = f'result_shuffled/plots/individual_clients/client_{client_id + 1}_accuracy_vs_rounds.png'
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"   ✓ Saved: client_{client_id + 1}_accuracy_vs_rounds.png")
    plt.close()

# ────────────────────────────────────────────
# Plot 2: All clients in one grid
# ────────────────────────────────────────────
print("\n2. Creating combined grid of all clients...")

cols = min(3, NUM_CLIENTS)
rows = math.ceil(NUM_CLIENTS / cols)

fig, axes = plt.subplots(rows, cols, figsize=(6*cols, 5*rows))

# Handle single row/column case
if NUM_CLIENTS == 1:
    axes = np.array([[axes]])
elif rows == 1:
    axes = axes.reshape(1, -1)
elif cols == 1:
    axes = axes.reshape(-1, 1)

fig.suptitle('Test Accuracy vs Communication Rounds (Adaptive Training)', 
             fontsize=18, fontweight='bold')

colors = plt.cm.tab10(np.linspace(0, 1, NUM_CLIENTS))

for client_id in range(NUM_CLIENTS):
    row = client_id // cols
    col = client_id % cols
    
    accuracies = [acc * 100 for acc in client_test_accuracies[f'client_{client_id}']]
    
    axes[row, col].plot(range(1, NUM_ROUNDS + 1), accuracies, 
                        marker='o', linewidth=2.5, markersize=8, 
                        color=colors[client_id])
    
    # Add accuracy values on points
    for i, acc in enumerate(accuracies):
        axes[row, col].text(i + 1, acc + 0.3, f'{acc:.1f}%', 
                           ha='center', fontsize=8, fontweight='bold')
    
    axes[row, col].set_title(f'Client {client_id + 1}', fontsize=14, fontweight='bold')
    axes[row, col].set_xlabel('Communication Round', fontsize=11)
    axes[row, col].set_ylabel('Test Accuracy (%)', fontsize=11)
    axes[row, col].grid(True, alpha=0.3)
    axes[row, col].set_xticks(range(1, NUM_ROUNDS + 1))
    axes[row, col].set_ylim([85, 100])
    
    # Add start and end accuracy text box
    start_acc = accuracies[0]
    end_acc = accuracies[-1]
    improvement = end_acc - start_acc
    total_epochs = sum(client_epochs_used[f'client_{client_id}'])
    axes[row, col].text(0.02, 0.02, 
                       f'Start: {start_acc:.1f}%\nEnd: {end_acc:.1f}%\nΔ: {improvement:+.1f}%\nEpochs: {total_epochs}',
                       transform=axes[row, col].transAxes,
                       bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7),
                       fontsize=8, verticalalignment='bottom')

# Hide empty subplots
total_subplots = rows * cols
for idx in range(NUM_CLIENTS, total_subplots):
    row = idx // cols
    col = idx % cols
    axes[row, col].axis('off')

plt.tight_layout()
save_path = 'result_shuffled/plots/all_clients_accuracy_vs_rounds_grid.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f"   ✓ Saved: all_clients_accuracy_vs_rounds_grid.png")
plt.close()

# ────────────────────────────────────────────
# Plot 3: All clients on same graph (OVERLAY)
# ────────────────────────────────────────────
print("\n3. Creating overlay plot of all clients...")

plt.figure(figsize=(12, 7))

colors = plt.cm.tab10(np.linspace(0, 1, NUM_CLIENTS))

for client_id in range(NUM_CLIENTS):
    accuracies = [acc * 100 for acc in client_test_accuracies[f'client_{client_id}']]
    plt.plot(range(1, NUM_ROUNDS + 1), accuracies, 
             marker='o', linewidth=2.5, markersize=8, 
             color=colors[client_id],
             label=f'Client {client_id + 1}')

plt.title('Test Accuracy vs Communication Rounds (All Clients)', 
          fontsize=16, fontweight='bold')
plt.xlabel('Communication Round', fontsize=13)
plt.ylabel('Test Accuracy (%)', fontsize=13)
plt.legend(loc='lower right', fontsize=11)
plt.grid(True, alpha=0.3)
plt.xticks(range(1, NUM_ROUNDS + 1))
plt.ylim([85, 100])

save_path = 'result_shuffled/plots/all_clients_accuracy_overlay.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f"   ✓ Saved: all_clients_accuracy_overlay.png")
plt.close()

# ────────────────────────────────────────────
# Plot 4: Average accuracy with standard deviation
# ────────────────────────────────────────────
print("\n4. Creating average client accuracy plot...")

plt.figure(figsize=(12, 7))

avg_accuracies = []
std_accuracies = []
for round_num in range(NUM_ROUNDS):
    round_accs = [client_test_accuracies[f'client_{i}'][round_num] * 100 for i in range(NUM_CLIENTS)]
    avg_accuracies.append(np.mean(round_accs))
    std_accuracies.append(np.std(round_accs))

plt.plot(range(1, NUM_ROUNDS + 1), avg_accuracies, 
         marker='s', linewidth=3, markersize=10, 
         color='red', label=f'Average ({NUM_CLIENTS} clients)')

plt.fill_between(range(1, NUM_ROUNDS + 1), 
                 np.array(avg_accuracies) - np.array(std_accuracies),
                 np.array(avg_accuracies) + np.array(std_accuracies),
                 alpha=0.2, color='red')

for i, acc in enumerate(avg_accuracies):
    plt.text(i + 1, acc + 0.5, f'{acc:.2f}%', 
            ha='center', fontsize=10, fontweight='bold')

plt.title('Average Test Accuracy vs Communication Rounds', 
          fontsize=16, fontweight='bold')
plt.xlabel('Communication Round', fontsize=13)
plt.ylabel('Test Accuracy (%)', fontsize=13)
plt.legend(loc='lower right', fontsize=11)
plt.grid(True, alpha=0.3)
plt.xticks(range(1, NUM_ROUNDS + 1))
plt.ylim([85, 100])

save_path = 'result_shuffled/plots/average_accuracy_vs_rounds.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f"   ✓ Saved: average_accuracy_vs_rounds.png")
plt.close()

# ────────────────────────────────────────────
# Plot 5: Epochs used heatmap
# ────────────────────────────────────────────
print("\n5. Creating epochs used heatmap...")

plt.figure(figsize=(10, 6))

epochs_matrix = np.zeros((NUM_CLIENTS, NUM_ROUNDS))
for client_id in range(NUM_CLIENTS):
    for round_num in range(NUM_ROUNDS):
        epochs_matrix[client_id, round_num] = client_epochs_used[f'client_{client_id}'][round_num]

plt.imshow(epochs_matrix, cmap='YlOrRd', aspect='auto')
plt.colorbar(label='Epochs Used')

for i in range(NUM_CLIENTS):
    for j in range(NUM_ROUNDS):
        plt.text(j, i, f'{int(epochs_matrix[i, j])}', 
                ha='center', va='center', fontsize=12, fontweight='bold')

plt.title('Epochs Used per Client per Communication Round', fontsize=16, fontweight='bold')
plt.xlabel('Communication Round', fontsize=13)
plt.ylabel('Client', fontsize=13)
plt.xticks(range(NUM_ROUNDS), range(1, NUM_ROUNDS + 1))
plt.yticks(range(NUM_CLIENTS), range(1, NUM_CLIENTS + 1))

save_path = 'result_shuffled/plots/epochs_used_heatmap.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f"   ✓ Saved: epochs_used_heatmap.png")
plt.close()

# ────────────────────────────────────────────
# Plot 6: Average epochs per round (bar chart)
# ────────────────────────────────────────────
print("\n6. Creating average epochs per round bar chart...")

plt.figure(figsize=(12, 6))

avg_epochs_per_round = []
for round_num in range(NUM_ROUNDS):
    round_epochs = [client_epochs_used[f'client_{i}'][round_num] for i in range(NUM_CLIENTS)]
    avg_epochs_per_round.append(np.mean(round_epochs))

bars = plt.bar(range(1, NUM_ROUNDS + 1), avg_epochs_per_round, color='steelblue', edgecolor='black')

for i, epochs in enumerate(avg_epochs_per_round):
    plt.text(i + 1, epochs + 0.5, f'{epochs:.1f}', ha='center', fontsize=11, fontweight='bold')

plt.title('Average Epochs Used per Communication Round', fontsize=16, fontweight='bold')
plt.xlabel('Communication Round', fontsize=13)
plt.ylabel('Average Epochs', fontsize=13)
plt.xticks(range(1, NUM_ROUNDS + 1))
plt.grid(True, alpha=0.3, axis='y')

save_path = 'result_shuffled/plots/avg_epochs_per_round.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f"   ✓ Saved: avg_epochs_per_round.png")
plt.close()

# ────────────────────────────────────────────
# Plot 7: Epochs vs Accuracy improvement per round
# ────────────────────────────────────────────
print("\n7. Creating epochs vs accuracy improvement plot...")

fig, ax1 = plt.subplots(figsize=(12, 6))

# Average epochs per round
ax1.set_xlabel('Communication Round', fontsize=13)
ax1.set_ylabel('Average Epochs Used', fontsize=13, color='steelblue')
ax1.bar(range(1, NUM_ROUNDS + 1), avg_epochs_per_round, color='steelblue', alpha=0.7, label='Epochs')
ax1.tick_params(axis='y', labelcolor='steelblue')

# Average accuracy per round (secondary axis)
ax2 = ax1.twinx()
ax2.set_ylabel('Average Accuracy (%)', fontsize=13, color='red')
ax2.plot(range(1, NUM_ROUNDS + 1), avg_accuracies, 
         marker='o', linewidth=3, markersize=10, color='red', label='Accuracy')
ax2.tick_params(axis='y', labelcolor='red')

plt.title('Epochs Used vs Accuracy per Communication Round', fontsize=16, fontweight='bold')
plt.xticks(range(1, NUM_ROUNDS + 1))

fig.tight_layout()
save_path = 'result_shuffled/plots/epochs_vs_accuracy.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f"   ✓ Saved: epochs_vs_accuracy.png")
plt.close()

print("\n" + "="*70)
print("ALL PLOTS GENERATED SUCCESSFULLY!")
print("="*70)
print(f"\nConfiguration: {NUM_CLIENTS} clients, {NUM_ROUNDS} rounds (Adaptive)")
print("\nPlots saved in:")
print(f"  - result_shuffled/plots/individual_clients/ ({NUM_CLIENTS} individual plots)")
print("  - result_shuffled/plots/all_clients_accuracy_vs_rounds_grid.png")
print("  - result_shuffled/plots/all_clients_accuracy_overlay.png")
print("  - result_shuffled/plots/average_accuracy_vs_rounds.png")
print("  - result_shuffled/plots/epochs_used_heatmap.png")
print("  - result_shuffled/plots/avg_epochs_per_round.png")
print("  - result_shuffled/plots/epochs_vs_accuracy.png")
print("="*70)

GENERATING PLOTS

1. Creating individual client test accuracy plots...
   ✓ Saved: client_1_accuracy_vs_rounds.png
   ✓ Saved: client_2_accuracy_vs_rounds.png
   ✓ Saved: client_3_accuracy_vs_rounds.png
   ✓ Saved: client_4_accuracy_vs_rounds.png
   ✓ Saved: client_5_accuracy_vs_rounds.png

2. Creating combined grid of all clients...
   ✓ Saved: all_clients_accuracy_vs_rounds_grid.png

3. Creating overlay plot of all clients...
   ✓ Saved: all_clients_accuracy_overlay.png

4. Creating average client accuracy plot...
   ✓ Saved: average_accuracy_vs_rounds.png

5. Creating epochs used heatmap...
   ✓ Saved: epochs_used_heatmap.png

6. Creating average epochs per round bar chart...
   ✓ Saved: avg_epochs_per_round.png

7. Creating epochs vs accuracy improvement plot...
   ✓ Saved: epochs_vs_accuracy.png

ALL PLOTS GENERATED SUCCESSFULLY!

Configuration: 5 clients, 10 rounds (Adaptive)

Plots saved in:
  - result_shuffled/plots/individual_clients/ (5 individual plots)
  - result_shuffled/p

In [11]:
# Cell 11: Save results
# ============================================
# SAVE RESULTS
# ============================================

# Save accuracy and epochs data
results_data = {
    'client_test_accuracies': client_test_accuracies,
    'client_epochs_used': client_epochs_used,
    'num_clients': NUM_CLIENTS,
    'num_rounds': NUM_ROUNDS,
    'min_epochs': MIN_EPOCHS,
    'max_epochs': MAX_EPOCHS,
    'patience': PATIENCE,
    'target_improvement': TARGET_IMPROVEMENT
}
np.save('results/adaptive_training_results.npy', results_data)
print(f"✓ Results saved to: results/adaptive_training_results.npy")

# Save global model
model_save_path = 'results/federated_global_model_adaptive.h5'
global_model.save(model_save_path)
print(f"✓ Final global model saved to: {model_save_path}")

print("\n✅ FEDERATED LEARNING (ADAPTIVE) COMPLETE!")
print(f"   Clients: {NUM_CLIENTS}")
print(f"   Rounds: {NUM_ROUNDS}")
print(f"   Total Epochs Used: {sum(all_total_epochs)}")
print(f"   Average Final Accuracy: {np.mean(all_final_accs):.2f}%")

✓ Results saved to: results/adaptive_training_results.npy
✓ Final global model saved to: results/federated_global_model_adaptive.h5

✅ FEDERATED LEARNING (ADAPTIVE) COMPLETE!
   Clients: 5
   Rounds: 10
   Total Epochs Used: 669
   Average Final Accuracy: 96.03%
